## Transfer learning adaptado para cualquier capa

In [ ]:
import h5py
import pandas as pd
import numpy as np

# Rutas a los archivos
topology_nuevo_path = '/ceph01/projects/AGRamirez_misc/carpeta_alberto_moreno/datos/transfer_learning/real_transfer_learning/topology.csv'
topology_old_path = 'topology.csv'
bestweights_old_path = 'bestweights_job.h5'
bestweights_new_path = '/ceph01/projects/AGRamirez_misc/carpeta_alberto_moreno/datos/transfer_learning/bestweights_job.h5'

# Especificar las capas a transferir (por índices, e.g., 0 para LocallyDirected_0)
capas_a_transferir = [0]  

# Cargar las topologías
topology_nuevo = pd.read_csv(topology_nuevo_path)
topology_old = pd.read_csv(topology_old_path)

def obtener_nombres_capas(capa):
    """
    Dada una capa 'n', retorna los nombres de las capas para el kernel y el bias.
    """
    layer0_name = f'layer{2 * capa}_name'      # Para el kernel de LocallyDirected_n
    layer1_name = f'layer{2 * capa + 1}_name'  # Para el bias de LocallyDirected_n
    return layer0_name, layer1_name

def obtener_topology_unique(topology, capas):
    """
    Para cada capa en 'capas', elimina duplicados basados en 'layer{2n+1}_name' para asegurar mapeo único de biases.
    """
    topology_unique = {}
    for capa in capas:
        layer1_name = f'layer{2 * capa + 1}_name'
        if layer1_name in topology.columns:
            topology_unique[capa] = topology.drop_duplicates(subset=[layer1_name])
        else:
            raise ValueError(f"La columna {layer1_name} no existe en la topología antigua.")
    return topology_unique

# Eliminar duplicados en 'layer{2n+1}_name' para cada capa a transferir
topology_old_unique = obtener_topology_unique(topology_old, capas_a_transferir)

# Crear mapeos de SNPs y Biases por capa
snp_kernel_mapping = {}
bias_mapping_old_unique = {}

for capa in capas_a_transferir:
    layer0_name, layer1_name = obtener_nombres_capas(capa)
    
    # Verificar que las columnas existen en la topología antigua
    if layer0_name not in topology_old.columns or layer1_name not in topology_old.columns:
        raise ValueError(f"Las columnas {layer0_name} o {layer1_name} no existen en la topología antigua.")
    
    # Crear mapeo SNP -> índice para el kernel de esta capa
    snp_kernel_mapping[capa] = {snp: idx for idx, snp in enumerate(topology_old[layer0_name])}
    
    # Crear mapeo Bias -> índice único para el bias de esta capa
    bias_mapping_old_unique[capa] = {bias: idx for idx, bias in enumerate(topology_old_unique[capa][layer1_name])}

# Identificar SNPs y Biases coincidentes por capa
snps_coincidentes_por_capa = {}
bias_coincidentes_por_capa = {}

for capa in capas_a_transferir:
    layer0_name, layer1_name = obtener_nombres_capas(capa)
    
    snps_nuevo_numbers = set(topology_nuevo[layer0_name])
    bias_nuevo_names = set(topology_nuevo[layer1_name])
    
    snps_old_numbers = set(topology_old[layer0_name])
    bias_old_names = set(topology_old_unique[capa][layer1_name])
    
    snps_coincidentes = snps_nuevo_numbers.intersection(snps_old_numbers)
    bias_coincidentes = bias_nuevo_names.intersection(bias_old_names)
    
    snps_coincidentes_por_capa[capa] = snps_coincidentes
    bias_coincidentes_por_capa[capa] = bias_coincidentes

# Cargar los pesos originales
with h5py.File(bestweights_old_path, 'r') as f:
    # Cargar kernels y biases de las capas LocallyDirected especificadas
    kernels_old = {}
    biases_old = {}
    for capa in capas_a_transferir:
        layer_name = f'LocallyDirected_{capa}'
        try:
            kernels_old[capa] = f[f'model_weights/{layer_name}/{layer_name}/kernel:0'][:]
            biases_old[capa] = f[f'model_weights/{layer_name}/{layer_name}/bias:0'][:]
        except KeyError as e:
            raise KeyError(f"No se encontró la capa {layer_name} en el archivo de pesos antiguos: {e}")
    
    # Cargar parámetros de Batch Normalization correspondientes
    moving_mean_old = {}
    moving_variance_old = {}
    for capa in capas_a_transferir:
        bn_layer = f'batch_normalization_{capa}' if capa > 0 else 'batch_normalization'
        try:
            moving_mean_old[capa] = f[f'model_weights/{bn_layer}/{bn_layer}/moving_mean:0'][:]
            moving_variance_old[capa] = f[f'model_weights/{bn_layer}/{bn_layer}/moving_variance:0'][:]
        except KeyError as e:
            raise KeyError(f"No se encontró la capa de batch normalization {bn_layer} en el archivo de pesos antiguos: {e}")
    
    # Cargar pesos de la capa de salida (Dense), NO TIENE QUE SER NECESARIO, EN NUESTRO CASO SI PQ EL TRANSFER ERA HASTA LA ULTIMA CAPA
    try:
        output_kernel_old = f['model_weights/output_layer/output_layer/kernel:0'][:]
        output_bias_old = f['model_weights/output_layer/output_layer/bias:0'][:]  # Copia directa del bias de salida
    except KeyError as e:
        raise KeyError(f"No se encontró la capa de salida 'output_layer' en el archivo de pesos antiguos: {e}")

# Inicializar estructuras para los nuevos pesos
kernel_nuevo = {}
bias_nuevo = {}
moving_mean_nuevo = {}
moving_variance_nuevo = {}

# Transferencia de pesos por capa
for capa in capas_a_transferir:
    layer0_name, layer1_name = obtener_nombres_capas(capa)
    
    # Obtener SNPs y Biases coincidentes para esta capa
    snps_coincidentes = snps_coincidentes_por_capa[capa]
    bias_coincidentes = bias_coincidentes_por_capa[capa]
    
    # Obtener mapeos específicos para esta capa
    snp_mapping = snp_kernel_mapping[capa]
    bias_mapping = bias_mapping_old_unique[capa]
    
    # Obtener índices de SNPs coincidentes
    indices_coincidentes_snps = [snp_mapping[snp] for snp in snps_coincidentes if snp in snp_mapping]
    
    # Extraer kernel y bias antiguos
    kernel_antiguo = kernels_old[capa]
    bias_antiguo = biases_old[capa]
    
    # Crear nuevo kernel solo con SNPs coincidentes
    kernel_nuevo[capa] = kernel_antiguo[indices_coincidentes_snps]
    
    # Inicializar nuevo bias con copia del bias antiguo
    bias_nuevo[capa] = np.copy(bias_antiguo)
    
    # Identificar SNPs faltantes en la nueva topología
    snps_faltantes = set(topology_old[layer0_name]).difference(set(topology_nuevo[layer0_name]))
    print(f"Capas {capa} - Número de SNPs faltantes calculados: {len(snps_faltantes)}")
    
    # Ajustar biases para SNPs faltantes
    for layer1, group in topology_old[topology_old[layer0_name].isin(snps_faltantes)].groupby(layer1_name):
        snps_grupo = set(group[layer0_name])
        # Sumar los kernels correspondientes a los SNPs faltantes
        total_peso_faltante = np.sum([kernel_antiguo[snp_mapping[snp]] for snp in snps_grupo if snp in snp_mapping], axis=0)
        
        if layer1 in bias_mapping:
            idx_bias = bias_mapping[layer1]
            if idx_bias < len(bias_nuevo[capa]):
                bias_nuevo[capa][idx_bias] += total_peso_faltante
            else:
                print(f"Capas {capa} - Índice fuera de rango: idx_bias={idx_bias}, no se puede ajustar.")
        else:
            print(f"Capas {capa} - Layer1_name '{layer1}' no está en bias_mapping_old_unique")
    
    # Recalcular parámetros de Batch Normalization usando el nuevo kernel
    moving_mean_nuevo[capa] = np.mean(kernel_nuevo[capa], axis=0)
    moving_variance_nuevo[capa] = np.var(kernel_nuevo[capa], axis=0)

# Ajustar la capa output_layer (Dense) solo si es necesario
# En este caso, no es necesario ajustar la capa de salida
output_kernel_nuevo = output_kernel_old.copy()  # Mantener el kernel de salida original
output_bias_nuevo = np.copy(output_bias_old)    # Copia directa del bias de salida sin ajustes

# Guardar los nuevos pesos en un archivo HDF5
with h5py.File(bestweights_new_path, 'w') as f:
    # Guardar los nuevos pesos de cada capa LocallyDirected especificada
    for capa in capas_a_transferir:
        layer_name = f'LocallyDirected_{capa}'
        f.create_dataset(f'model_weights/{layer_name}/{layer_name}/kernel:0', data=kernel_nuevo[capa])
        f.create_dataset(f'model_weights/{layer_name}/{layer_name}/bias:0', data=bias_nuevo[capa])
        
        # Guardar los nuevos pesos de batch normalization
        bn_layer = f'batch_normalization_{capa}' if capa > 0 else 'batch_normalization'
        f.create_dataset(f'model_weights/{bn_layer}/{bn_layer}/moving_mean:0', data=moving_mean_nuevo[capa])
        f.create_dataset(f'model_weights/{bn_layer}/{bn_layer}/moving_variance:0', data=moving_variance_nuevo[capa])
        
        # Crear la capa de activación vacía correspondiente a esta capa LocallyDirected
        activation_layer = 'activation' if capa == 0 else f'activation_{capa}'
        f.create_dataset(f'model_weights/{activation_layer}/{activation_layer}', data=[])
    
    # Guardar los nuevos pesos de la capa Dense (output_layer) sin modificaciones, NO TIENE PORQUÉ SER NECESARIO
    f.create_dataset('model_weights/output_layer/output_layer/kernel:0', data=output_kernel_nuevo)
    f.create_dataset('model_weights/output_layer/output_layer/bias:0', data=output_bias_nuevo)  # Copia directa del bias de salida
    
    # Mantener la estructura de activaciones vacías para capas no transferidas (si es necesario)
    # Esto asegura que todas las capas de activación existentes en la topología antigua estén presentes
    activaciones_existentes = [col for col in topology_old.columns if col.startswith('activation')]
    for activation in activaciones_existentes:
        activation_path = f'model_weights/{activation}/{activation}'
        if activation not in f['model_weights']:
            f.create_dataset(activation_path, data=[])
    
    print(f"Nuevos archivos de pesos guardados en: {bestweights_new_path}")

# Función para imprimir la estructura de capas en un archivo HDF5
def imprimir_estructura_hdf5(ruta_archivo):
    """
    Imprime la estructura de capas y datasets en el archivo HDF5 especificado.
    """
    with h5py.File(ruta_archivo, 'r') as f:
        print(f"\nEstructura de capas en '{ruta_archivo}':")
        def imprimir_grupo(grupo, indent=0):
            for key in grupo:
                item = grupo[key]
                print("    " * indent + f"- {key}")
                if isinstance(item, h5py.Group):
                    imprimir_grupo(item, indent + 1)
                elif isinstance(item, h5py.Dataset):
                    print("    " * (indent + 1) + f"Dataset: {key}, Shape: {item.shape}, Dtype: {item.dtype}")
        
        imprimir_grupo(f)

# Imprimir la estructura del nuevo archivo HDF5
imprimir_estructura_hdf5(bestweights_new_path)

print("Proceso de transfer learning completado exitosamente.")


# Transfer especifico que apliqué

In [ ]:
import h5py
import pandas as pd
import numpy as np

# Rutas a los archivos
topology_nuevo_path = '/ceph01/projects/AGRamirez_misc/carpeta_alberto_moreno/datos/transfer_learning/real_transfer_learning/topology.csv'
topology_old_path = '/ceph01/projects/AGRamirez_misc/campos/Alberto/tot_Federico/data_Frederico/hg38_genes/topology.csv'
bestweights_old_path = '/ceph01/projects/AGRamirez_misc/campos/Alberto/tot_Federico/data_Frederico/bestweights_job.h5'
bestweights_new_path = '/ceph01/projects/AGRamirez_misc/carpeta_alberto_moreno/datos/transfer_learning/real_transfer_learning/bestweights_job.h5'

# Cargar las topologías
topology_nuevo = pd.read_csv(topology_nuevo_path)
topology_old = pd.read_csv(topology_old_path)

# Eliminar duplicados en `layer1_name` para asegurar mapeo único
topology_old_unique = topology_old.drop_duplicates(subset=['layer1_name'])

# Crear mapeo entre los SNPs y los bias en `topology_old_unique`
snp_kernel_mapping = {snp: idx for idx, snp in enumerate(topology_old['layer0_name'])}
bias_mapping_old_unique = {bias: idx for idx, bias in enumerate(topology_old_unique['layer1_name'])}

# Identificar SNPs y biases coincidentes entre la nueva topología y la vieja topología
snps_nuevo_numbers = set(topology_nuevo['layer0_name'])
bias_nuevo_names = set(topology_nuevo['layer1_name'])
snps_old_numbers = set(topology_old['layer0_name'])
bias_old_names = set(topology_old_unique['layer1_name'])
snps_coincidentes = snps_nuevo_numbers.intersection(snps_old_numbers)
bias_coincidentes = bias_nuevo_names.intersection(bias_old_names)

# Cargar los pesos originales
with h5py.File(bestweights_old_path, 'r') as f:
    # LocallyDirected_0 pesos
    kernel_old = f['model_weights/LocallyDirected_0/LocallyDirected_0/kernel:0'][:]
    bias_old = f['model_weights/LocallyDirected_0/LocallyDirected_0/bias:0'][:]
    
    # Batch normalization pesos
    moving_mean_old = f['model_weights/batch_normalization/batch_normalization/moving_mean:0'][:]
    moving_variance_old = f['model_weights/batch_normalization/batch_normalization/moving_variance:0'][:]
    
    # Output layer (Dense) pesos
    output_kernel_old = f['model_weights/output_layer/output_layer/kernel:0'][:]
    output_bias_old = f['model_weights/output_layer/output_layer/bias:0'][:]  # Copia directa del bias de salida

# Crear un nuevo kernel solo con los SNPs coincidentes en LocallyDirected_0
indices_coincidentes_snps = [snp_kernel_mapping[snp] for snp in snps_coincidentes]
kernel_nuevo = kernel_old[indices_coincidentes_snps]

# Inicializar bias_nuevo con una copia de bias_old para LocallyDirected_0
bias_nuevo = np.copy(bias_old)

# Identificar los SNPs faltantes en la nueva topología que no tienen coincidencias en la vieja topología
snps_faltantes = snps_old_numbers.difference(snps_nuevo_numbers)
print("Número de SNPs faltantes calculados:", len(snps_faltantes))

# Ajustar los bias de LocallyDirected_0 para los SNPs faltantes
for layer1_name, group in topology_old[topology_old['layer0_name'].isin(snps_faltantes)].groupby('layer1_name'):
    snps_grupo = set(group['layer0_name'])
    total_peso_faltante = np.sum([kernel_old[snp_kernel_mapping[snp]] for snp in snps_grupo])

    if layer1_name in bias_mapping_old_unique:
        idx_bias = bias_mapping_old_unique[layer1_name]
        if idx_bias < len(bias_nuevo):
            bias_nuevo[idx_bias] += total_peso_faltante
        else:
            print(f"Índice fuera de rango: idx_bias={idx_bias}, no se puede ajustar.")
    else:
        print(f"Layer1_name {layer1_name} no está en bias_mapping_old_unique")

# Ajustar la capa output_layer (Dense)
output_kernel_nuevo = output_kernel_old[:len(indices_coincidentes_snps), :]  # Ajuste del kernel para los SNPs coincidentes
output_bias_nuevo = np.copy(output_bias_old)  # Copia directa del bias de salida sin ajustes

# Aquí usamos el kernel_nuevo para recalcular los parámetros de batch normalization
moving_mean_nuevo = np.mean(kernel_nuevo, axis=0)
moving_variance_nuevo = np.var(kernel_nuevo, axis=0)

# Guardar los nuevos pesos en un archivo HDF5
with h5py.File(bestweights_new_path, 'w') as f:
    # Guardar los nuevos pesos del kernel y bias de LocallyDirected_0
    f.create_dataset('model_weights/LocallyDirected_0/LocallyDirected_0/kernel:0', data=kernel_nuevo)
    f.create_dataset('model_weights/LocallyDirected_0/LocallyDirected_0/bias:0', data=bias_nuevo)

    # Guardar los nuevos pesos de batch normalization (solo los relacionados con LocallyDirected_0)
    f.create_dataset('model_weights/batch_normalization/batch_normalization/moving_mean:0', data=moving_mean_nuevo)
    f.create_dataset('model_weights/batch_normalization/batch_normalization/moving_variance:0', data=moving_variance_nuevo)

    # Guardar los nuevos pesos del kernel y bias de la capa Dense (output_layer)
    f.create_dataset('model_weights/output_layer/output_layer/kernel:0', data=output_kernel_nuevo)
    f.create_dataset('model_weights/output_layer/output_layer/bias:0', data=output_bias_nuevo)  # Copia directa del bias de salida

    # Crear la capa de activación vacía (sin parámetros), puede que no sea necesario pero de esta manera nos aseguramos la estructura que queremos
    f.create_dataset('model_weights/activation/activation', data=[])
    f.create_dataset('model_weights/activation_1/activation_1', data=[])

print("Nuevo archivo de pesos guardado en:", bestweights_new_path)
